In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import deeplake
import numpy as np
from sklearn.model_selection import train_test_split


2023-09-18 21:35:12.552371: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-18 21:35:12.765533: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-18 21:35:12.767385: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-18 21:35:14.056562: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Загрузка данных
ds = deeplake.load('hub://activeloop/not-mnist-small')

|

Opening dataset in read-only mode as you don't have write permissions.


\

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/not-mnist-small



\

hub://activeloop/not-mnist-small loaded successfully.



In [3]:
ds

Dataset(path='hub://activeloop/not-mnist-small', read_only=True, tensors=['images', 'labels'])

In [5]:
# Преобразование данных в numpy массивы
images = np.array(ds['images'])
labels = np.array(ds['labels'])

# Разбиение данных на тренировочный и тестовый наборы
notmnist_train, notmnist_test, _, _ = train_test_split(images, labels, test_size=0.2, random_state=42)

In [32]:
# Загрузка и предобработка данных
(fashion_train, fashion_train_labels), (fashion_test, fashion_test_labels) = tf.keras.datasets.fashion_mnist.load_data()

# Нормализация данных
fashion_train = fashion_train.astype('float32') / 255.
fashion_test = fashion_test.astype('float32') / 255.

# Архитектура автоэнкодера
input_img = Input(shape=(28, 28))
x = Flatten()(input_img)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
encoded = Dense(128, activation='relu')(x)

x = Dense(128, activation='relu')(encoded)
x = Dense(28*28, activation='softmax')(x)
decoded = Reshape((28, 28))(x)

autoencoder = Model(input_img, decoded)

# Обучение автоэнкодера на notMNIST
optimizer = Adam (learning_rate = 0.000001)
autoencoder.compile(optimizer=optimizer, loss='mse')
autoencoder.fit(notmnist_train, notmnist_train, epochs=50, batch_size=512, validation_data=(notmnist_test, notmnist_test))

# Создание модели для Fashion MNIST на основе весов кодировщика
encoder = Model(input_img, encoded)
for layer in encoder.layers:
    layer.trainable = False

x = Dense(128, activation='relu')(encoded)
predictions = Dense(10, activation='softmax')(x)

fashion_model = Model(input_img, predictions)
fashion_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
fashion_model.fit(fashion_train, fashion_train_labels, epochs=50, batch_size=256, validation_data=(fashion_test, fashion_test_labels))

# Предсказание
predictions = fashion_model.predict(fashion_test)


Epoch 1/50
30/30 [==============================] - 2s 35ms/step - loss: 25436.3926 - val_loss: 25464.2051
Epoch 2/50
30/30 [==============================] - 1s 31ms/step - loss: 25436.3945 - val_loss: 25464.2051
Epoch 3/50
29/30 [============================>.] - ETA: 0s - loss: 25439.7617

KeyboardInterrupt: 

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model


# Загрузка и предобработка данных
(fashion_train, fashion_train_labels), (fashion_test, fashion_test_labels) = tf.keras.datasets.fashion_mnist.load_data()

# Нормализация данных
fashion_train = fashion_train.astype('float32') / 255.
fashion_test = fashion_test.astype('float32') / 255.

# Архитектура сверточного автоэнкодера
input_img = Input(shape=(28, 28, 1))  # Fashion MNIST - черно-белые изображения

# Энкодер
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Декодер
x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

# Сборка автоэнкодера
optimizer = Adam (learning_rate = 0.001)
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer=optimizer, loss='binary_crossentropy')

# Предполагается, что fashion_train и fashion_test имеют размерность (num_samples, 28, 28)
# Мы просто изменяем размерность данных для соответствия форме (num_samples, 28, 28, 1)
fashion_train = fashion_train.reshape(-1, 28, 28, 1)
fashion_test = fashion_test.reshape(-1, 28, 28, 1)

autoencoder.fit(fashion_train, fashion_train,
                epochs=50,
                batch_size=256,
                validation_data=(fashion_test, fashion_test))

Epoch 1/50
235/235 [==============================] - 78s 327ms/step - loss: 0.3156 - val_loss: 0.2812
Epoch 2/50
235/235 [==============================] - 77s 329ms/step - loss: 0.2755 - val_loss: 0.2741
Epoch 3/50
235/235 [==============================] - 82s 348ms/step - loss: 0.2704 - val_loss: 0.2705
Epoch 4/50
235/235 [==============================] - 81s 345ms/step - loss: 0.2674 - val_loss: 0.2678
Epoch 5/50
235/235 [==============================] - 81s 344ms/step - loss: 0.2646 - val_loss: 0.2660
Epoch 6/50
235/235 [==============================] - 81s 346ms/step - loss: 0.2618 - val_loss: 0.2625
Epoch 7/50
235/235 [==============================] - 81s 344ms/step - loss: 0.2597 - val_loss: 0.2608
Epoch 8/50
235/235 [==============================] - 80s 342ms/step - loss: 0.2582 - val_loss: 0.2594
Epoch 9/50
235/235 [==============================] - 79s 338ms/step - loss: 0.2571 - val_loss: 0.2592
Epoch 10/50
235/235 [==============================] - 80s 339ms/step - l

KeyboardInterrupt: 

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D


(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Измените размер изображений и преобразуйте в трехканальный формат
train_images = np.repeat(np.resize(train_images, (train_images.shape[0], 224, 224)), 3, axis=-1).reshape(train_images.shape[0], 224, 224, 3)
test_images = np.repeat(np.resize(test_images, (test_images.shape[0], 224, 224)), 3, axis=-1).reshape(test_images.shape[0], 224, 224, 3)

# Нормализуйте изображения
train_images = train_images.astype('float32') / 255.
test_images = test_images.astype('float32') / 255.


In [ ]:

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Добавьте новые верхние слои
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Заморозьте слои базовой модели VGG16
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, batch_size=32, epochs=5, validation_data=(test_images, test_labels))


In [ ]:
# Разморозка верхних слоев
for layer in model.layers[:15]:
    layer.trainable = False
for layer in model.layers[15:]:
    layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, batch_size=32, epochs=5, validation_data=(test_images, test_labels))
